In [ ]:
%cd /home/jaeheonshim/music-vibes

/home/jaeheonshim/music-vibes


In [2]:
import torch
import numpy as np
from torch import nn, autocast
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import random_split, DataLoader, Subset
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from collections import defaultdict

from vibenet import labels
from vibenet.dataset import FMAWaveformDataset
from vibenet.models.student import EfficientNetRegressor
from vibenet.train_utils import compute_losses, compute_metrics

device = 'cuda'

In [3]:
train_ds = FMAWaveformDataset('data/preprocessed/waveforms_distill_train')
test_ds = FMAWaveformDataset('data/preprocessed/waveforms_distill_test')

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=8, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

print(len(train_ds))

90437


In [4]:
net = EfficientNetRegressor()
net = net.to(device)

NUM_EPOCHS = 50

optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-5)

In [5]:
writer = SummaryWriter()

scaler = torch.amp.GradScaler()

def train():
    global global_step
    
    net.train()
    
    train_losses = []

    with tqdm(train_dl, desc='Training') as pbar:
        for data, label in pbar:
            data, label = data.to(device).float(), label.to(device).float()
            optimizer.zero_grad()

            with autocast(device_type='cuda', dtype=torch.bfloat16):
                pred = net(data)
                
                loss_total = 0.0
                losses = compute_losses(pred, label)
                for k, l in losses.items():
                    writer.add_scalar(f"train/loss/{k}", l, global_step)
                    loss_total += l
                
            scaler.scale(loss_total).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()

            train_losses.append(loss_total.item())
            
            mean_loss = np.mean(train_losses)
            writer.add_scalar(f"train/loss", mean_loss, global_step)
            pbar.set_postfix({'loss': f"{mean_loss:.4f}"})
            
            global_step += 1

def validate():
    global global_step
    
    net.eval()
    
    preds = defaultdict(list)
    targets = []

    with tqdm(test_dl, desc='Validation') as pbar:
        with torch.inference_mode():
            for data, label in pbar:
                data, label = data.to(device).float(), label.to(device).float()

                pred = net(data)
                for l, p in pred.items():
                    preds[l].append(p.detach().cpu())

                targets.append(label.detach().cpu())

    preds = {k: torch.cat(preds[k], dim=0) for k in preds.keys()}
    targets = torch.cat(targets, dim=0)

    losses = compute_losses(preds, targets)
    loss_total = 0.0
            
    for k,l in losses.items():
        loss_total += l

    writer.add_scalar(f"eval/loss", loss_total, global_step)

    metrics = compute_metrics(preds, targets)

    return metrics, loss_total.item()

Teacher model metrics (for reference)

Loss: 2.1312

[acousticness]
  logloss : 0.5195

[danceability]
  mse     : 0.0179
  rmse    : 0.1338
  mae     : 0.1061
  pearson : 0.7192
  ccc     : 0.7056
  r2      : 0.5025

[energy]
  mse     : 0.0362
  rmse    : 0.1902
  mae     : 0.1468
  pearson : 0.8009
  ccc     : 0.7911
  r2      : 0.5287

[instrumentalness]
  logloss : 0.5282

[liveness]
  logloss : 0.4789

[speechiness]
  mse     : 0.0108
  rmse    : 0.1041
  mae     : 0.0552
  pearson : 0.7217
  ccc     : 0.7205
  r2      : 0.4706

[valence]
  mse     : 0.0631
  rmse    : 0.2512
  mae     : 0.1958
  pearson : 0.6587
  ccc     : 0.6437
  r2      : 0.1492

In [6]:
global_step = 0

best_loss = float("inf")

for epoch in range(NUM_EPOCHS):
    print(f"\nEpoch {epoch+1}:")
    
    writer.add_scalar(f"epoch", epoch, global_step)

    train()
    metrics, loss_total = validate()

    print(f"Loss: {loss_total:.4f}")
    for task, stats in metrics.items():
        print(f"\n[{task}]")
        max_name_len = max(len(k) for k in stats.keys())
        for name, val in stats.items():        
            writer.add_scalar(f"eval/{task}/{name}", val, global_step)
    
            if isinstance(val, float):
                print(f"  {name:<{max_name_len}} : {val:.4f}")
            else:
                print(f"  {name:<{max_name_len}} : {val}")

    if loss_total < best_loss:
        best_loss = loss_total
        torch.save({'state_dict': net.state_dict()},
                   'checkpoints/efficientnet_best.pt')
        print('Saved new best model')
        
    scheduler.step()
    
    global_step += 1
    
writer.flush()


Epoch 1:


Training:   0%|                                                                                                                                                                                              | 0/707 [00:00<?, ?it/s]

Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:45<00:00,  5.48it/s]


Loss: 1.6879

[acousticness]
  logloss : 0.5108

[danceability]
  mse     : 0.0057
  rmse    : 0.0752
  mae     : 0.0606
  pearson : 0.8739
  ccc     : 0.8422
  r2      : 0.7292

[energy]
  mse     : 0.0154
  rmse    : 0.1242
  mae     : 0.0983
  pearson : 0.9229
  ccc     : 0.9111
  r2      : 0.7977

[instrumentalness]
  logloss : 0.5148

[liveness]
  logloss : 0.4540

[speechiness]
  mse     : 0.0038
  rmse    : 0.0613
  mae     : 0.0361
  pearson : 0.8659
  ccc     : 0.8282
  r2      : 0.7308

[valence]
  mse     : 0.0162
  rmse    : 0.1272
  mae     : 0.0951
  pearson : 0.9045
  ccc     : 0.8950
  r2      : 0.7900
Saved new best model

Epoch 2:


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:44<00:00,  5.59it/s]


...

Loss: 1.5081

[acousticness]
  logloss : 0.4892

[danceability]
  mse     : 0.0019
  rmse    : 0.0433
  mae     : 0.0335
  pearson : 0.9543
  ccc     : 0.9527
  r2      : 0.9101

[energy]
  mse     : 0.0044
  rmse    : 0.0666
  mae     : 0.0490
  pearson : 0.9707
  ccc     : 0.9706
  r2      : 0.9418

[instrumentalness]
  logloss : 0.4917

[liveness]
  logloss : 0.4498

[speechiness]
  mse     : 0.0012
  rmse    : 0.0344
  mae     : 0.0198
  pearson : 0.9570
  ccc     : 0.9550
  r2      : 0.9152

[valence]
  mse     : 0.0065
  rmse    : 0.0809
  mae     : 0.0569
  pearson : 0.9570
  ccc     : 0.9568
  r2      : 0.9150

Epoch 49:


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:44<00:00,  5.58it/s]


Loss: 1.5079

[acousticness]
  logloss : 0.4894

[danceability]
  mse     : 0.0019
  rmse    : 0.0432
  mae     : 0.0333
  pearson : 0.9544
  ccc     : 0.9534
  r2      : 0.9108

[energy]
  mse     : 0.0044
  rmse    : 0.0667
  mae     : 0.0492
  pearson : 0.9707
  ccc     : 0.9707
  r2      : 0.9417

[instrumentalness]
  logloss : 0.4916

[liveness]
  logloss : 0.4498

[speechiness]
  mse     : 0.0012
  rmse    : 0.0343
  mae     : 0.0201
  pearson : 0.9570
  ccc     : 0.9562
  r2      : 0.9157

[valence]
  mse     : 0.0066
  rmse    : 0.0812
  mae     : 0.0573
  pearson : 0.9573
  ccc     : 0.9570
  r2      : 0.9143

Epoch 50:


Validation: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:44<00:00,  5.61it/s]


Loss: 1.5073

[acousticness]
  logloss : 0.4891

[danceability]
  mse     : 0.0019
  rmse    : 0.0432
  mae     : 0.0333
  pearson : 0.9543
  ccc     : 0.9529
  r2      : 0.9107

[energy]
  mse     : 0.0044
  rmse    : 0.0666
  mae     : 0.0491
  pearson : 0.9709
  ccc     : 0.9708
  r2      : 0.9417

[instrumentalness]
  logloss : 0.4916

[liveness]
  logloss : 0.4498

[speechiness]
  mse     : 0.0012
  rmse    : 0.0343
  mae     : 0.0199
  pearson : 0.9569
  ccc     : 0.9558
  r2      : 0.9156

[valence]
  mse     : 0.0065
  rmse    : 0.0808
  mae     : 0.0569
  pearson : 0.9572
  ccc     : 0.9572
  r2      : 0.9153
Saved new best model
